In [1]:
import re
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset, DataLoader

import numpy as np
import pandas as pd
from tqdm import tqdm
from copy import deepcopy
from collections import Counter
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

In [2]:
class LabelRecoDataset(Dataset):
    def __init__(
        self,
        text,
        label,
        vocab=None,
        is_train=True,
        max_length=100
    ) -> None:
        super().__init__()

        self.text = text
        self.label = torch.tensor(label, dtype=torch.float32)
        self.max_length = max_length

        if is_train:
            self.vocab = self.build_vocab(self.text)
        else:
            if isinstance(vocab, str):
                pass
            elif vocab:
                self.vocab = vocab

        
    def build_vocab(self, text):    
        counter = Counter()
        for line in text:
            counter.update(list(line))
        
        word_to_index = {word: idx+2 for idx, (word, _) in enumerate(counter.items())}
        word_to_index['<PAD>'] = 0
        word_to_index['<UNKNOWN>'] = 1
        index_to_word = {idx: word for word, idx in word_to_index.items()}
        
        return word_to_index

    def word2idx(self, text):
        return [self.vocab.get(t, 1) for t in list(text)[-self.max_length:]]

    def __getitem__(self, index):
        return torch.tensor(self.word2idx(self.text[index]), dtype=torch.long), self.label[index]
    
    def __len__(self):
        return len(self.label)

In [3]:
def load_data(file_path, test_size=0.1):
    df = pd.read_csv(file_path).fillna('')
    train_text, test_text, train_label, test_label = train_test_split(df['text'], df['label'], test_size=test_size, random_state=42, stratify=df['label'])
    return train_text.tolist(), test_text.tolist(), train_label.tolist(), test_label.tolist()

In [4]:
def collate_fn(batch):
    text, label = zip(*batch)
    text = [item.clone().detach() for item in text]
    text = pad_sequence(text, batch_first=True, padding_value=0)
    label = torch.tensor(label, dtype=torch.float32)
    return text, label

In [5]:
def get_loader(file_path, max_length=100, batch_size=32):
    train_text, test_text, train_label, test_label = load_data(file_path)
    train_dataset = LabelRecoDataset(train_text, train_label, max_length=max_length)
    test_dataset = LabelRecoDataset(test_text, test_label, vocab=train_dataset.vocab, is_train=False, max_length=max_length)
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)
    
    return train_loader, test_loader, train_dataset.vocab

In [6]:
class DetectModel(nn.Module):
    def __init__(
        self,
        vocab_size=50,
        input_size=64,
        hidden_size=128,
        output_size=1,
        dropout_p=.5
    ) -> None:
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, input_size)
        self.rnn = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=4,
            batch_first=True,
            dropout=dropout_p,
            bidirectional=True
        )
        
        self.linear = nn.Linear(hidden_size * 2, output_size)
        
        
    def forward(self, x):
        x = self.embedding(x)
        z, _ = self.rnn(x)
        z = z[:, -1, :]
        y = self.linear(z)
        
        return y

In [7]:
class Trainer(nn.Module):
    def __init__(self, model, crit, optimizer, device=None):
        super().__init__()
        
        if not device:
            self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        else: self.device = device
        self.model = model.to(self.device)
        self.crit = crit.to(self.device)
        self.optimizer = optimizer

        self.train_history = []
        self.valid_history = []

    def _train(self, train_loader):
        train_loss = 0

        for X_i, y_i in tqdm(train_loader, desc='Train'):
            X_i = X_i.to(self.device)
            y_i = y_i.to(self.device)

            y_hat_i = self.model(X_i)
            loss = self.crit(y_hat_i.squeeze(), y_i)

            self.optimizer.zero_grad()
            loss.backward()

            self.optimizer.step()
            train_loss += float(loss)

        return train_loss / len(train_loader)

    def _valid(self, valid_loader):
        valid_loss = 0

        with torch.no_grad():
            for X_i, y_i in tqdm(valid_loader, desc='Validate'):
                X_i = X_i.to(self.device)
                y_i = y_i.to(self.device)

                y_hat_i = self.model(X_i)
                loss = self.crit(y_hat_i.squeeze(), y_i) 

                valid_loss += float(loss)

        return valid_loss / len(valid_loader)


    def train(
        self, 
        train_loader,
        valid_loader, 
        epochs=10, 
        print_interval=1, 
        early_stop=5,
        figsize=(20, 10)
    ):        
        best_model = None
        lowest_loss = np.inf
        lowest_epoch = 0


        for epoch in range(1, epochs + 1):
            train_loss = self._train(train_loader)
            valid_loss = self._valid(valid_loader)

            self.train_history += [train_loss]
            self.valid_history += [valid_loss]

            if valid_loss <= lowest_loss:
                best_model = deepcopy(self.model.state_dict())
                lowest_loss = valid_loss
                lowest_epoch = epoch

            else:
                if (epoch - lowest_epoch) >= early_stop:
                    print("Early Stop at Epoch {epoch}. Lowest Loss {lowest_loss} at {lowest_epoch} epoch".format(
                        epoch=epoch,
                        lowest_loss=round(lowest_loss, 5),
                        lowest_epoch=lowest_epoch
                    ))
                    break

            if epoch % print_interval == 0:
                print("Epoch {epoch}: train_loss={train_loss}, valid_loss={valid_loss}, lowest_loss={lowest_loss}".format(
                    epoch=epoch,
                    train_loss=round(train_loss, 5),
                    valid_loss=round(valid_loss, 5),
                    lowest_loss=round(lowest_loss, 5)
                ))

        print('The best validstion loss from epoch={epoch}, loss={lowest_loss}'.format(
            epoch=lowest_epoch,
            lowest_loss=round(lowest_loss, 5)
        ))

        self.model.load_state_dict(best_model)

In [8]:
max_length = 100

In [9]:
train_loader, test_loader, vocab = get_loader('./data/cost_detect_dataset.csv', max_length=max_length, batch_size=256)

/home/jcwee/anaconda3/envs/torch/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/home/jcwee/anaconda3/envs/torch/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


In [11]:
df = pd.read_csv('./data/cost_detect_dataset2.csv')

In [12]:
df

,Unnamed: 0,text,label
0,0,홍대에서 데이트 하기로 한 주말\n진짜 간만에 홍대\n전에 항상 세우던 공영주차장이...,0
1,1,Vied INTF 주택상업공간 HAN SALSN 쟁 반 집 反 절염,0
2,2,오픈전 지나갈때는 사람이 없었는데\n오픈시간 12시가 다가오면 #홍대고기집맛집 앞에...,0
3,3,재반집 JAENG BAN JIB8292 OPEN12:00 PM CLOSE 02:00...,0
4,4,나도 홍대 #고기집 쟁반집 앞에서 기다려야지!,0
...,...,...,...
639533,639533,NaN,0
639534,639534,NaN,0
639535,639535,은근 자리도 넓고 안주도 저렴한데 맛있고 하이볼 종류도 많아서 내가 조와하는곳이징 ...,0
639536,639536,9! 월마감날 함께한 블랙그레이즈드라떼 가을이왔구나 !,0


In [14]:
df.label.value_counts() / df.label.value_counts().sum()

label
0    0.991178
1    0.008822
Name: count, dtype: float64

In [10]:
model = DetectModel(
    vocab_size=len(vocab),
    input_size=128,
    hidden_size=256,
    output_size=1,
    dropout_p=0.5
)

In [11]:
pos_weight_value = 50.0
pos_weight = torch.tensor([pos_weight_value])

optimizer = optim.Adam(model.parameters())
crit = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

In [12]:
trainer = Trainer(
    model,
    crit,
    optimizer
)

In [13]:
trainer.train(
    train_loader,
    test_loader,
    epochs=20
)

Validate: 100%|██████████| 250/250 [00:07<00:00, 34.33it/s]


Epoch 1: train_loss=0.28337, valid_loss=0.1316, lowest_loss=0.1316


Validate: 100%|██████████| 250/250 [00:07<00:00, 34.24it/s]


Epoch 2: train_loss=0.0965, valid_loss=0.10038, lowest_loss=0.10038


Validate: 100%|██████████| 250/250 [00:07<00:00, 34.30it/s]


...

Epoch 14: train_loss=0.01692, valid_loss=0.08638, lowest_loss=0.06128


Validate: 100%|██████████| 250/250 [00:07<00:00, 33.93it/s]


Epoch 15: train_loss=0.01458, valid_loss=0.07379, lowest_loss=0.06128


Validate: 100%|██████████| 250/250 [00:07<00:00, 33.91it/s]

Early Stop at Epoch 16. Lowest Loss 0.06128 at 11 epoch
The best validstion loss from epoch=11, loss=0.06128


In [14]:
y = []
y_h = []
X = []
for X_i, y_i in test_loader:
    y += y_i
    with torch.no_grad():
        X_i = X_i.to(torch.device('cuda:0'))
        y_h += trainer.model(X_i).cpu()
        
        X += list(X_i.cpu())

In [15]:
y = torch.stack(y).numpy()
y

array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)

In [16]:
y_preds = F.sigmoid(torch.stack(y_h).squeeze())
y_preds = (y_preds >= 0.5).float().numpy()
y_preds

array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)

In [17]:
confusion_matrix(y, y_preds)

array([[63219,   171],
       [   11,   553]])

In [2]:
63219+171+11+553

63954

In [3]:
(63219 + 553) / 63954

0.9971542045845452

In [4]:
63219 / (63219 + 171)

0.9973024136299101

In [6]:
553 / (171+553)

0.7638121546961326

In [18]:
import json
from glob import glob

In [19]:
files = glob('./data/semi_struct_ocr/*.json')

In [20]:
def extract_contents(row: dict):
    contents = []
    
    for content in row['contents']:
        if content['info'] not in ['text', 'img']: continue
            
        text = content.get('ocr_text', content.get('content'))
        contents.append(text)
        
    return contents

def contents_to_tensor(contents: list, vocab: dict, max_length=100):
    vectors = [torch.tensor([vocab.get(c, 1) for c in content][-max_length:], dtype=torch.long) for content in contents]
    tensor = pad_sequence(vectors, batch_first=True, padding_value=0)
    return tensor

In [29]:
result = []
for file in files:
    with open(file, 'r', encoding='utf-8') as f:
        blog = json.load(f)
    for article in tqdm(blog, desc=file.split('/')[-1].split('.')[0]):
        contents = extract_contents(article)
        if len(contents) == 0:
            result.append({
                'id': article['id'],
                'reason': '',
                'ocr_label': 0,
            })     
            continue
            
        tensor = contents_to_tensor(contents, vocab, max_length=max_length)

        with torch.no_grad():
            X = tensor.to(torch.device('cuda:0'))
            y_h = trainer.model(X)
            y_preds = F.sigmoid(y_h.cpu().squeeze())
            y_preds = (y_preds >= 0.5).float().numpy()

            if y_preds.reshape(-1)[-5:].sum() > 0:
                result.append({
                    'id': article['id'],
                    'reason': '\n'.join(contents[-5:]),
                    'ocr_label': 1,
                })
            else:
                result.append({
                    'id': article['id'],
                    'reason': '\n'.join(contents[-5:]),
                    'ocr_label': 0,
                })     

영등포 데이트 맛집: 100%|██████████| 927/927 [00:07<00:00, 121.39it/s]


In [44]:
ocr_labels = pd.DataFrame(result)
ocr_labels[ocr_labels.reason.str.contains('#')].reason.apply(lambda x: x[-40:])

0                포스팅은 업체로부터 서비스만을 제공받아 솔직하게 작성된 포스팅 입니다*
1              \n본포스팅은해당업체로부터 소정의제품을제공받았지만 솔직하게 작성 하였습니다
2        \n\n\n\n\n\n\n* 위 포스팅은 소정의 원고료를 제공 받아 작성되었습니다 *
4               본포스팅은해당 업체로부터 소정의원고료를자원받았지만 솔직하게작성 하였습니다
6               의여왕을 통해본 업체에서 제품또는 서비스를제공받아 솔직하게 작성하였습니다
                              ...                       
11073           추 #서로이웃추천 #맛집추천 #맛집추천해주세요 #추천해주시면가서사먹을께요
11075          려요!\n결론 : 타임스퀘어에 위치한, 편백찜 세트메뉴가 가성비 있는 곳!
11076           깨비굴 #도깨비굴폭립 #여의도데이트추천 #영등포데이트추천 #서울데이트추천
11078           트맛집 #서울맛집 #문래동맛집추천 #문래데이트맛집추천 #문래분위기맛집추천
11080           #여의나루삼겹살 #샛강 #샛강맛집 #샛강데이트 #샛강데이트맛집 #샛강저녁
Name: reason, Length: 4486, dtype: object

In [31]:
labels = pd.read_csv('./data/label.csv', index_col=0)
labels

,id,search_word,title,url,blog_name,date,content,content_len,content_hash_cnt,article_hash_cnt,...,total_post,adpost_yn,link_cnt,map_yn,video_yn,phone_yn,image_cnt,image_ocr_text,new_con_ocr,label
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
0,shuenmama.223027650182,홍대 회식 맛집,홍대 고기집 데이트 쟁반한상 삼겹살 맛집 회식 쟁반집8292,https://blog.naver.com/shuenmama/223027650182,shuen,20230225,홍대에서데이트하기로한주말진짜간만에홍대전에항상세우던공영주차장이막혀있어서근처유료주차장에...,3094,10,12,...,523,3,0,1,1,0,35,바기어로닐든정반우쟁반발구이막창쟁반소시지수제숫불닭발사이드된장말이밥김치말이밥된장찌개김치...,홍대에서데이트하기로한주말진짜간만에홍대전에항상세우던공영주차장이막혀있어서근처유료주차장에...,1
1,rosepink1974.223153722255,홍대 회식 맛집,홍대삼겹살 청년화로 1987 이베리코 연남동고기집 회식맛집,https://blog.naver.com/rosepink1974/223153722255,예쁜 달코미의 단맛 인생,20230712,청년화로1987서울마포구동교로2191층청년화로1987홍대입구역3번출구에서278m월금...,2345,9,9,...,418,3,0,1,1,0,45,새나라아니아스써시새에라별는폴같은쥐방충이깝싸고였는부워프팔백타고번폰한강일당소고기같은큰내...,청년화로1987서울마포구동교로2191층청년화로1987홍대입구역3번출구에서278m월금...,0
2,mou25.223209216526,홍대 회식 맛집,"합정 맛집 홍대 회식장소로 딱, 느낌 있는 소고기 고깃집...",https://blog.naver.com/mou25/223209216526,생애 기록장,20230912,매번느끼는거지만회식장소하나는기가막히게섭외하는울주임님얼마전에도팀원들끼리고기먹을일있었는...,2904,12,12,...,554,3,0,1,0,0,34,NaN,매번느끼는거지만회식장소하나는기가막히게섭외하는울주임님얼마전에도팀원들끼리고기먹을일있었는...,1
3,lulu_l.223118434610,홍대 회식 맛집,"홍대 맛집 합정 갈비가 부드러운 소고기집 연막탄 회식, 데이트...",https://blog.naver.com/lulu_l/223118434610,안나의 일상공유,20230602,홍대소고기맛집연막탄남자친구가맛있는고깃집을알고있다길해합정맛집으로인정받고있는연막탄에다녀...,2810,0,7,...,222,3,0,2,0,0,30,토고노이드히그일바얼리얼맛집연탄불구위아세고기류때표메뉴원갈비시그니쳐똥갈비한우육회사이드벡...,홍대소고기맛집연막탄남자친구가맛있는고깃집을알고있다길해합정맛집으로인정받고있는연막탄에다녀...,1
4,ruston_.223161590597,홍대 회식 맛집,"홍대회식, 육즙 폭발하는 소고기 맛집 '일편등심 홍대본점'",https://blog.naver.com/ruston_/223161590597,로빈이 토끼란 사실을 알고있었나?,20230720,안녕하세요LoLCake입니다최근이직을준비하는동료의축하파티를하는의미에서특별한자리를마련...,2299,2,2,...,573,3,0,1,0,0,18,합니민세요서정말잘구워크는게주시고구워그리다간이나티션갈이구프라이빗하게자리마다파구워주십니...,안녕하세요LoLCake입니다최근이직을준비하는동료의축하파티를하는의미에서특별한자리를마련...,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10484,suitbull.222112955908,영등포 데이트 맛집,"영등포 돈까스 맛집, 타임스퀘어 데이트 ::간코::",https://blog.naver.com/suitbull/222112955908,SUITBULL(숯불)의 미식 그리고 풍류,20201011,안녕하세요미식을즐기는악식가(樂食家)SUITBULL(숯불)입니다영등포타임스퀘어를참자주...,1642,0,11,...,231,3,0,2,1,1,26,영등포돈까스타임스반반카츠우심결때연두부튀김래드동심돈카츠간코둔카츠특벌한전분과단빅칠을활용...,안녕하세요미식을즐기는악식가(樂食家)SUITBULL(숯불)입니다영등포타임스퀘어를참자주...,1
10485,dusal0652.222413131913,영등포 데이트 맛집,[영등포 문래동] 문래창작촌에서도 유명한 양키통닭,https://blog.naver.com/dusal0652/222413131913,기록하는 습관,20210628,날씨가좋은어느날문래에서일어난일ㅎㅎㅎㅎㅎ친구랑같이가기로한양키통닭지인이일하고있는가게여서...,1996,12,12,...,171,3,0,2,1,0,25,다비자재배녀다져이버시도타시서시재네기트데이치지히가게시시시어기백이네개비래해채시지시타타시...,날씨가좋은어느날문래에서일어난일ㅎㅎㅎㅎㅎ친구랑같이가기로한양키통닭지인이일하고있는가게여서...,0
10486,shinesparis.221509975993,영등포 데이트 맛집,영등포역 맛집 '럭치미' 인도요리 커리 완전맛있어~!,https://blog.naver.com/shinesparis/221509975993,귀여운 서소남매,20190410,인도요리를좋아하는편이라신랑이랑가끔먹으러가곤한다영등포역에위치한인도요리커리맛집인럭치미(...,2350,0,11,...,911,3,0,2,0,0,0,NaN,인도요리를좋아하는편이라신랑이랑가끔먹으러가곤한다영등포역에위치한인도요리커리맛집인럭치미(...,0


In [49]:
df = pd.merge(
    labels,
    ocr_labels,
    on='id'
)
df.loc[df[df.label == 1].index, 'ocr_label'] = 1
df

,id,search_word,title,url,blog_name,date,content,content_len,content_hash_cnt,article_hash_cnt,...,link_cnt,map_yn,video_yn,phone_yn,image_cnt,image_ocr_text,new_con_ocr,label,reason,ocr_label
0,shuenmama.223027650182,홍대 회식 맛집,홍대 고기집 데이트 쟁반한상 삼겹살 맛집 회식 쟁반집8292,https://blog.naver.com/shuenmama/223027650182,shuen,20230225,홍대에서데이트하기로한주말진짜간만에홍대전에항상세우던공영주차장이막혀있어서근처유료주차장에...,3094,10,12,...,0,1,1,0,35,바기어로닐든정반우쟁반발구이막창쟁반소시지수제숫불닭발사이드된장말이밥김치말이밥된장찌개김치...,홍대에서데이트하기로한주말진짜간만에홍대전에항상세우던공영주차장이막혀있어서근처유료주차장에...,1,\n그리고 대망의 도톰한 삼겹살\n고기를 잘 못구워도 겉바 속촉 하게 구울수 밖에없...,1
1,rosepink1974.223153722255,홍대 회식 맛집,홍대삼겹살 청년화로 1987 이베리코 연남동고기집 회식맛집,https://blog.naver.com/rosepink1974/223153722255,예쁜 달코미의 단맛 인생,20230712,청년화로1987서울마포구동교로2191층청년화로1987홍대입구역3번출구에서278m월금...,2345,9,9,...,0,1,1,0,45,새나라아니아스써시새에라별는폴같은쥐방충이깝싸고였는부워프팔백타고번폰한강일당소고기같은큰내...,청년화로1987서울마포구동교로2191층청년화로1987홍대입구역3번출구에서278m월금...,0,일반 돼지고기보다 부드럽고 쫄깃한\n이베리코 스페셜이었어요.\n\n저희는 정말 만족...,1
2,mou25.223209216526,홍대 회식 맛집,"합정 맛집 홍대 회식장소로 딱, 느낌 있는 소고기 고깃집...",https://blog.naver.com/mou25/223209216526,생애 기록장,20230912,매번느끼는거지만회식장소하나는기가막히게섭외하는울주임님얼마전에도팀원들끼리고기먹을일있었는...,2904,12,12,...,0,1,0,0,34,NaN,매번느끼는거지만회식장소하나는기가막히게섭외하는울주임님얼마전에도팀원들끼리고기먹을일있었는...,1,https://postfiles.pstatic.net/MjAyMzA5MTJfMTYw...,1
3,lulu_l.223118434610,홍대 회식 맛집,"홍대 맛집 합정 갈비가 부드러운 소고기집 연막탄 회식, 데이트...",https://blog.naver.com/lulu_l/223118434610,안나의 일상공유,20230602,홍대소고기맛집연막탄남자친구가맛있는고깃집을알고있다길해합정맛집으로인정받고있는연막탄에다녀...,2810,0,7,...,0,2,0,0,30,토고노이드히그일바얼리얼맛집연탄불구위아세고기류때표메뉴원갈비시그니쳐똥갈비한우육회사이드벡...,홍대소고기맛집연막탄남자친구가맛있는고깃집을알고있다길해합정맛집으로인정받고있는연막탄에다녀...,1,이번에 다녀 온 곳은 저도 남자친구도\n모두 만족할 수 있는 곳이었는데요.\n다음 ...,1
4,ruston_.223161590597,홍대 회식 맛집,"홍대회식, 육즙 폭발하는 소고기 맛집 '일편등심 홍대본점'",https://blog.naver.com/ruston_/223161590597,로빈이 토끼란 사실을 알고있었나?,20230720,안녕하세요LoLCake입니다최근이직을준비하는동료의축하파티를하는의미에서특별한자리를마련...,2299,2,2,...,0,1,0,0,18,합니민세요서정말잘구워크는게주시고구워그리다간이나티션갈이구프라이빗하게자리마다파구워주십니...,안녕하세요LoLCake입니다최근이직을준비하는동료의축하파티를하는의미에서특별한자리를마련...,1,"LoLCake\n아 참, 투쁠 한우의 최상의 맛을 위해 새우살과 눈꽃살치는 한정수량...",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10573,suitbull.222112955908,영등포 데이트 맛집,"영등포 돈까스 맛집, 타임스퀘어 데이트 ::간코::",https://blog.naver.com/suitbull/222112955908,SUITBULL(숯불)의 미식 그리고 풍류,20201011,안녕하세요미식을즐기는악식가(樂食家)SUITBULL(숯불)입니다영등포타임스퀘어를참자주...,1642,0,11,...,0,2,1,1,26,영등포돈까스타임스반반카츠우심결때연두부튀김래드동심돈카츠간코둔카츠특벌한전분과단빅칠을활용...,안녕하세요미식을즐기는악식가(樂食家)SUITBULL(숯불)입니다영등포타임스퀘어를참자주...,1,"바삭한 두부 튀김 속에 부드러운 연두부의 식감이 간장, 마요네즈 소스와 함께하니 가...",1
10574,dusal0652.222413131913,영등포 데이트 맛집,[영등포 문래동] 문래창작촌에서도 유명한 양키통닭,https://blog.naver.com/dusal0652/222413131913,기록하는 습관,20210628,날씨가좋은어느날문래에서일어난일ㅎㅎㅎㅎㅎ친구랑같이가기로한양키통닭지인이일하고있는가게여서...,1996,12,12,...,0,2,1,0,25,다비자재배녀다져이버시도타시서시재네기트데이치지히가게시시시어기백이네개비래해채시지시타타시...,날씨가좋은어느날문래에서일어난일ㅎㅎㅎㅎㅎ친구랑같이가기로한양키통닭지인이일하고있는가게여서...,0,"\n2명이서 먹었는데 많이 남았음...\n배불러....ㅎ\n\n""(딸랑딸랑~~~) ...",1
10575,shinesparis.221509975993,영등포 데이트 맛집,영등포역 맛집 '럭치미' 인도요리 커리 완전맛있어~!,https://blog.naver.com/shinesparis/221509975993,귀여운 서소남매,20190410,인도요리를좋아하는편이라신랑이랑가끔먹으러가곤한다영등포역에위치한인도요리커리맛집인럭치미(...,2350,0,11,...,0,2,0,0,0,NaN,인도요리를좋아하는편이라신랑이랑가끔먹으러가곤한다영등포역에위치한인도요리커리맛집인럭치미(...,0,\n배터졌지만 스타벅스가서\n신랑은 아이스 아메리카노 한잔~\n난 바닐라아이스크림을...,0
10576,sleeping_yulmu.223070079287,영등포 데이트 맛집,[서울/여의도] 블링 블링 빛이 나는 맛 좋은 삼겹살! '돈블랑...,https://blog.naver.com/sleeping_yulmu/22307007...,sleeping_day,20230410,예년보다벚꽃이일찍개화한올해친구들과꽃놀이는빼놓을수가없죠함께저녁을먹고여의도공원을걷기로했...,1745,40,30,...,0,1,0,0,12,퀴로종순대국밥특추가메뉴후식된장축재래된장찌개공깃밥주문시우가주문가능깊은푸에또몽득한식감더...,예년보다벚꽃이일찍개화한올해친구들과꽃놀이는빼놓을수가없죠함께저녁을먹고여의도공원을걷기로했...,0,"고기가 다 익었어요!\n너무 맛있어 보이죠!\n냠냠 열심히 먹었는데,\n양이 많지가...",0


In [50]:
diff = df[df.label != df.ocr_label]
diff

,id,search_word,title,url,blog_name,date,content,content_len,content_hash_cnt,article_hash_cnt,...,link_cnt,map_yn,video_yn,phone_yn,image_cnt,image_ocr_text,new_con_ocr,label,reason,ocr_label
1,rosepink1974.223153722255,홍대 회식 맛집,홍대삼겹살 청년화로 1987 이베리코 연남동고기집 회식맛집,https://blog.naver.com/rosepink1974/223153722255,예쁜 달코미의 단맛 인생,20230712,청년화로1987서울마포구동교로2191층청년화로1987홍대입구역3번출구에서278m월금...,2345,9,9,...,0,1,1,0,45,새나라아니아스써시새에라별는폴같은쥐방충이깝싸고였는부워프팔백타고번폰한강일당소고기같은큰내...,청년화로1987서울마포구동교로2191층청년화로1987홍대입구역3번출구에서278m월금...,0,일반 돼지고기보다 부드럽고 쫄깃한\n이베리코 스페셜이었어요.\n\n저희는 정말 만족...,1
8,aoekdud.223100176598,홍대 회식 맛집,홍대 맛집 가본 사람은 못잊는... 고깃집 연막탄 회식장소 강추,https://blog.naver.com/aoekdud/223100176598,치즈♡,20230512,회사사람들과함께다녀온음식점을소개해드릴까하는데이미유명한곳중하나라서알고있는분들이많으실수...,2812,7,7,...,0,1,1,0,29,구스나가이이,회사사람들과함께다녀온음식점을소개해드릴까하는데이미유명한곳중하나라서알고있는분들이많으실수...,0,\n여러분들도 조금 더 특별한 곳에서 회식을\n하고 싶거나 든든하게 식사를 하고 싶...,1
93,sungmin4815.223198983706,홍대 회식 맛집,[마포구맛집/홍대회식] 홍대 가성비 좋고 매장이 짱 넓은...,https://blog.naver.com/sungmin4815/223198983706,대충 살자(งᐖ)ว,20230831,두둥홍대에이런곳이있었다니질좋은소갈비살을먹을수있는홍대소고기맛집“홍대자갈집”을소개해보려...,1098,20,19,...,0,2,1,0,31,일수요일아해전너주문을확인해주세요벼비이에개,두둥홍대에이런곳이있었다니질좋은소갈비살을먹을수있는홍대소고기맛집“홍대자갈집”을소개해보려...,0,1\n\n\n녹차라떼로 입가심~~🌿🌿\n#홍대코스 #연남동코스 #홍대나들이\n#연남...,1
104,dlsdud2005.223204849372,홍대 회식 맛집,홍대/신촌 야들야들 족발맛집 회식장소추천 &lt;행당족발&gt;,https://blog.naver.com/dlsdud2005/223204849372,유래삣과 뿡빵뿡빵,20230907,유래삣과뿡빵뿡빵입니다친구와의만남때문에처음으로신촌에방문했는데요어느맛집에서회포를풀까하다...,1415,19,19,...,0,2,0,0,23,삐안녕하서특대잎빈앞발뒷발부대찌개뽀부더마나같맹,유래삣과뿡빵뿡빵입니다친구와의만남때문에처음으로신촌에방문했는데요어느맛집에서회포를풀까하다...,0,배달족발집 막국수의 경우 비빔장에 꾸덕꾸덕한 비주얼인데 이곳 행당족발 쟁반막국수는 ...,1
184,nbc2656.221431232195,홍대 회식 맛집,홍대 맛집 : 홍대 회식장소 방어맛집 경성다찌,https://blog.naver.com/nbc2656/221431232195,일상의 기억,20181231,영업시간:평일17:00-02:00주말17:00-03:00예약번호:02-332-870...,1032,8,8,...,0,2,0,1,0,NaN,영업시간:평일17:00-02:00주말17:00-03:00예약번호:02-332-870...,0,홍대맛집 홍대방어맛집 경성다찌\n싱싱한 방어를 실컷먹었어여\n\n홍대맛집 홍대방어맛...,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10555,sm3510enl.222119861205,영등포 데이트 맛집,[영등포역/타임스퀘어] '딘타이펑' (메뉴판 O) 민또먹,https://blog.naver.com/sm3510enl/222119861205,톨ఇ,20201019,안녕하세톨ఇ⋆떡볶이덕후⋆이지만다른거먹은민톨입니다💜민또먹(민톨이또먹음)오랜만에샤오롱바...,1489,28,28,...,0,2,0,0,32,영등포역타임스퀘어딘타이펑딘타히펑은루미티티다고기사오통바오역시으저본바오가어가게소사서게가...,안녕하세톨ఇ⋆떡볶이덕후⋆이지만다른거먹은민톨입니다💜민또먹(민톨이또먹음)오랜만에샤오롱바...,0,"우육면에는 엄청 큰 고기가 두덩이나,,👍🏻\n\n오랜만에 먹은 딘타이펑\n넘 맛있었...",1
10559,tweety228.222771166372,영등포 데이트 맛집,"[맛집/영등포] 화사가 먹었다던 그 곱창집,대한곱창!+화려한...",https://blog.naver.com/tweety228/222771166372,아리 햅삐일상 모음,20220613,저번에곱창얘기하다가곱창이급땡겨서바로영등포근처곱창집을찾아봄보니까나혼자산다에서화사가맛있...,805,16,16,...,0,2,0,0,9,맞있게준비좀익니다모눔버섯마채추가원대한자돌박이원그원염통구이원음로첫주문은인분부터가능주가...,저번에곱창얘기하다가곱창이급땡겨서바로영등포근처곱창집을찾아봄보니까나혼자산다에서화사가맛있...,0,화려한 불쇼를 보여주신다ㅋㅋㅋㅋ\n너무 화력이 세서 깜짝 놀랐음\n\n거의 좀만 더...,1
10566,qhrud2117.221676586975,영등포 데이트 맛집,[영등포조개찜] 조개 가득한 조개찜에 닭칼국수까지!...,https://blog.naver.com/qhrud2117/221676586975,엄바나 블로그,20191013,[영등포조개찜]조개가득한조개찜에닭칼국수까지로꼬로꼬조개찜진짜오랜만에먹어보는조개찜ㅋㅋㅋ...,2753,8,14,...,0,1,2,1,20,아,[영등포조개찜]조개가득한조개찜에닭칼국수까지로꼬로꼬조개찜진짜오랜만에먹어보는조개찜ㅋㅋㅋ...,0,\n조개찜을 그렇게 먹고 ㅋㅋㅋㅋ\n닭 살도 발라먹고 했는데도\n칼국수 배는 남아있...,1
10568,hhappyduck.223199063701,영등포 데이트 맛집,[서울] 당산 콩국수 맛집 '서민준 밀밭',https://blog.naver.com/hhappyduck/223199063701,"먹고 빼고, 그게 일상",20230831,당산콩국수맛집서민준밀밭🍜🍜🍜🍜🍜🍜🍜🍜🍜🍜🍜🍜운영시간:10:00-20:30🍜브레이크타...,1065,59,30,...,0,1,0,0,7,글버키원사리들깨스제비바지락기만두개검정콩물고전요이나막걸리노란콩물뜨축소맥주보무미이디멘을...,당산콩국수맛집서민준밀밭🍜🍜🍜🍜🍜🍜🍜🍜🍜🍜🍜🍜운영시간:10:00-20:30🍜브레이크타...,0,https:/Iblog navercom/hhappyduck\n테이블당 소금 => -...,1


In [60]:
AD_KEYWORDS = [
    "일부내돈내산",
    "지원을받아",
    "지원받아",
    "후원을받아",
    "후원받아",
    "원고료",
    "체험단",
    "제공받아",
    "제공받았",
    "제공서비스",
    "금전적지원",
    "사비로",
    "업체로부터",
    "협찬",
    "제품또는서비스를",
    "업체에서",
    "소정의",
    "고료를",
    "할인을",
    "레뷰",
    "강남오빠",
    "디너의여왕",
    "서울오빠",
    "이놀자데뷰"
]

HAN_REGEX = re.compile('[^가-힣]')

In [64]:
diff

,id,search_word,title,url,blog_name,date,content,content_len,content_hash_cnt,article_hash_cnt,...,link_cnt,map_yn,video_yn,phone_yn,image_cnt,image_ocr_text,new_con_ocr,label,reason,ocr_label
1,rosepink1974.223153722255,홍대 회식 맛집,홍대삼겹살 청년화로 1987 이베리코 연남동고기집 회식맛집,https://blog.naver.com/rosepink1974/223153722255,예쁜 달코미의 단맛 인생,20230712,청년화로1987서울마포구동교로2191층청년화로1987홍대입구역3번출구에서278m월금...,2345,9,9,...,0,1,1,0,45,새나라아니아스써시새에라별는폴같은쥐방충이깝싸고였는부워프팔백타고번폰한강일당소고기같은큰내...,청년화로1987서울마포구동교로2191층청년화로1987홍대입구역3번출구에서278m월금...,0,일반 돼지고기보다 부드럽고 쫄깃한\n이베리코 스페셜이었어요.\n\n저희는 정말 만족...,1
8,aoekdud.223100176598,홍대 회식 맛집,홍대 맛집 가본 사람은 못잊는... 고깃집 연막탄 회식장소 강추,https://blog.naver.com/aoekdud/223100176598,치즈♡,20230512,회사사람들과함께다녀온음식점을소개해드릴까하는데이미유명한곳중하나라서알고있는분들이많으실수...,2812,7,7,...,0,1,1,0,29,구스나가이이,회사사람들과함께다녀온음식점을소개해드릴까하는데이미유명한곳중하나라서알고있는분들이많으실수...,0,\n여러분들도 조금 더 특별한 곳에서 회식을\n하고 싶거나 든든하게 식사를 하고 싶...,1
93,sungmin4815.223198983706,홍대 회식 맛집,[마포구맛집/홍대회식] 홍대 가성비 좋고 매장이 짱 넓은...,https://blog.naver.com/sungmin4815/223198983706,대충 살자(งᐖ)ว,20230831,두둥홍대에이런곳이있었다니질좋은소갈비살을먹을수있는홍대소고기맛집“홍대자갈집”을소개해보려...,1098,20,19,...,0,2,1,0,31,일수요일아해전너주문을확인해주세요벼비이에개,두둥홍대에이런곳이있었다니질좋은소갈비살을먹을수있는홍대소고기맛집“홍대자갈집”을소개해보려...,0,1\n\n\n녹차라떼로 입가심~~🌿🌿\n#홍대코스 #연남동코스 #홍대나들이\n#연남...,1
104,dlsdud2005.223204849372,홍대 회식 맛집,홍대/신촌 야들야들 족발맛집 회식장소추천 &lt;행당족발&gt;,https://blog.naver.com/dlsdud2005/223204849372,유래삣과 뿡빵뿡빵,20230907,유래삣과뿡빵뿡빵입니다친구와의만남때문에처음으로신촌에방문했는데요어느맛집에서회포를풀까하다...,1415,19,19,...,0,2,0,0,23,삐안녕하서특대잎빈앞발뒷발부대찌개뽀부더마나같맹,유래삣과뿡빵뿡빵입니다친구와의만남때문에처음으로신촌에방문했는데요어느맛집에서회포를풀까하다...,0,배달족발집 막국수의 경우 비빔장에 꾸덕꾸덕한 비주얼인데 이곳 행당족발 쟁반막국수는 ...,1
184,nbc2656.221431232195,홍대 회식 맛집,홍대 맛집 : 홍대 회식장소 방어맛집 경성다찌,https://blog.naver.com/nbc2656/221431232195,일상의 기억,20181231,영업시간:평일17:00-02:00주말17:00-03:00예약번호:02-332-870...,1032,8,8,...,0,2,0,1,0,NaN,영업시간:평일17:00-02:00주말17:00-03:00예약번호:02-332-870...,0,홍대맛집 홍대방어맛집 경성다찌\n싱싱한 방어를 실컷먹었어여\n\n홍대맛집 홍대방어맛...,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10555,sm3510enl.222119861205,영등포 데이트 맛집,[영등포역/타임스퀘어] '딘타이펑' (메뉴판 O) 민또먹,https://blog.naver.com/sm3510enl/222119861205,톨ఇ,20201019,안녕하세톨ఇ⋆떡볶이덕후⋆이지만다른거먹은민톨입니다💜민또먹(민톨이또먹음)오랜만에샤오롱바...,1489,28,28,...,0,2,0,0,32,영등포역타임스퀘어딘타이펑딘타히펑은루미티티다고기사오통바오역시으저본바오가어가게소사서게가...,안녕하세톨ఇ⋆떡볶이덕후⋆이지만다른거먹은민톨입니다💜민또먹(민톨이또먹음)오랜만에샤오롱바...,0,"우육면에는 엄청 큰 고기가 두덩이나,,👍🏻\n\n오랜만에 먹은 딘타이펑\n넘 맛있었...",1
10559,tweety228.222771166372,영등포 데이트 맛집,"[맛집/영등포] 화사가 먹었다던 그 곱창집,대한곱창!+화려한...",https://blog.naver.com/tweety228/222771166372,아리 햅삐일상 모음,20220613,저번에곱창얘기하다가곱창이급땡겨서바로영등포근처곱창집을찾아봄보니까나혼자산다에서화사가맛있...,805,16,16,...,0,2,0,0,9,맞있게준비좀익니다모눔버섯마채추가원대한자돌박이원그원염통구이원음로첫주문은인분부터가능주가...,저번에곱창얘기하다가곱창이급땡겨서바로영등포근처곱창집을찾아봄보니까나혼자산다에서화사가맛있...,0,화려한 불쇼를 보여주신다ㅋㅋㅋㅋ\n너무 화력이 세서 깜짝 놀랐음\n\n거의 좀만 더...,1
10566,qhrud2117.221676586975,영등포 데이트 맛집,[영등포조개찜] 조개 가득한 조개찜에 닭칼국수까지!...,https://blog.naver.com/qhrud2117/221676586975,엄바나 블로그,20191013,[영등포조개찜]조개가득한조개찜에닭칼국수까지로꼬로꼬조개찜진짜오랜만에먹어보는조개찜ㅋㅋㅋ...,2753,8,14,...,0,1,2,1,20,아,[영등포조개찜]조개가득한조개찜에닭칼국수까지로꼬로꼬조개찜진짜오랜만에먹어보는조개찜ㅋㅋㅋ...,0,\n조개찜을 그렇게 먹고 ㅋㅋㅋㅋ\n닭 살도 발라먹고 했는데도\n칼국수 배는 남아있...,1
10568,hhappyduck.223199063701,영등포 데이트 맛집,[서울] 당산 콩국수 맛집 '서민준 밀밭',https://blog.naver.com/hhappyduck/223199063701,"먹고 빼고, 그게 일상",20230831,당산콩국수맛집서민준밀밭🍜🍜🍜🍜🍜🍜🍜🍜🍜🍜🍜🍜운영시간:10:00-20:30🍜브레이크타...,1065,59,30,...,0,1,0,0,7,글버키원사리들깨스제비바지락기만두개검정콩물고전요이나막걸리노란콩물뜨축소맥주보무미이디멘을...,당산콩국수맛집서민준밀밭🍜🍜🍜🍜🍜🍜🍜🍜🍜🍜🍜🍜운영시간:10:00-20:30🍜브레이크타...,0,https:/Iblog navercom/hhappyduck\n테이블당 소금 => -...,1


In [65]:
additional_idx = []
for i, r in diff[diff.label == 0].reason.items():
    t = HAN_REGEX.sub('', r)
    
    for kywd in AD_KEYWORDS:
        if kywd in t:
            additional_idx.append(i)
            break
            
additional_idx = [i for i in diff.index if i not in additional_idx]

In [67]:
df.loc[additional_idx, 'ocr_label'] = 0

In [71]:
df.drop('reason', axis=1).to_csv('./data/label_ocr_classifier_added.csv')